<a href="https://colab.research.google.com/github/Partha-deuri/learn_pytorch/blob/main/webScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup
import re

#initialize
globalCorpus = []
globalLinks = set()
globalVisited = set()

rootPageUrl = 'https://www.asomiyapratidin.in/'
globalLinks.add(rootPageUrl)

# helpers
skipLines = [
        '© asomiyapratidin 2024'
    ]
def printList(list):
    for item in list:
        print(item)
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

# get all links

def fetchAllLinks (url,nested=20):
    if nested == 0:
        return
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")
    res=soup.find_all('a')
    for item in res:
        link = item.get('href')
        if link not in globalLinks:
            if re.search("^"+rootPageUrl+".*", link):
                globalLinks.add(link)
                fetchAllLinks(link,nested-1)

fetchAllLinks(rootPageUrl)


#get data from one page
def fetchData(link):
    corpus = []
    tempReq = requests.get(link)
    tempSoup = BeautifulSoup(tempReq.content,"html.parser")
    tempRes = tempSoup.findAll('p')
    for p in tempRes:
        if p.text not in skipLines:
            corpus.append(p.text)
    return corpus


# get data from all pages
def  getAllData(links=[]):
    for link in links:
        if link not in globalVisited:
            globalVisited.add(link)
            data = fetchData(link)
            if len(data) >0:
                globalCorpus.append(data)

getAllData(globalLinks)

# printList(globalCorpus)

# Create file
def createFile(filename):
    f = open(filename+".txt", "w", encoding='utf-8')
    for doc in globalCorpus:
        for para in doc:
            para = para.split('।')
            for line in para:
                line  = re.sub("[a-zA-Z0-9#|/]",'',line)
                line = deEmojify(line)
                f.write(line)
                f.write("\n")
    f.close()

createFile("assamese_corpus")

print("\n\n\t\t***** Task Completed *****\n\n")



		***** Task Completed *****


